# Manipulação de Dados

Após a aquisição dos dados, geralmente há muito esforço envolvido na manipulação destes para o facilitar/melhorar o processamento. Assume-se Python 3 no sistema Unix, e a preparação é simples

```bash
sudo pip3 install numpy pandas
```

## Numpy

O [Numpy](https://numpy.org/) fornece [ferramentas eficientes](https://medium.com/ensina-ai/entendendo-a-biblioteca-numpy-4858fde63355) para manipulação em memória de dados. Isso é feito pela "reestruturação" da forma como os vetores homogêneos multidimensionais são tratados.

In [1]:
num_items = 1000

python_array = range(num_items)
%timeit -n 100 [i**2 for i in python_array]

100 loops, best of 5: 203 µs per loop


In [2]:
import numpy as np

numpy_array = np.arange(num_items)
%timeit -n 100 numpy_array**2

The slowest run took 35.34 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 2.22 µs per loop


In [3]:
a = np.array([[0, 1], [2, 3]])
b = np.array([[0, 1], [-2, -3]])

# Operações [geralmente] são realizadas em cada elemento
print('\nProduto simples')
print(a * b)

print('\nProduto matricial')
print(a @ b) # a.dot(b)


Produto simples
[[ 0  1]
 [-4 -9]]

Produto matricial
[[-2 -3]
 [-6 -7]]


In [4]:
print('Fusão horizontal')
print(np.hstack((a, b)))

print('\nFusão vertical')
print(np.vstack((a, b)))

Fusão horizontal
[[ 0  1  0  1]
 [ 2  3 -2 -3]]

Fusão vertical
[[ 0  1]
 [ 2  3]
 [ 0  1]
 [-2 -3]]


In [5]:
print('Valores')
a = np.arange(10)**2
print(a)

i = np.array([1, 1, 3, 8, 5])
print('\nÍndices')
print(i)

print('\nIndexação')
print(a[i])

Valores
[ 0  1  4  9 16 25 36 49 64 81]

Índices
[1 1 3 8 5]

Indexação
[ 1  1  9 64 25]


In [6]:
import random

nums = [x for x in range(1, 13)]
random.shuffle(nums)

a = np.array(nums).reshape(3, 4)
print('Valores')
print(a)

multiplos_de_4 = (a % 4 == 0)
print('\nResultado do teste item a item')
print(multiplos_de_4)

print('\n... que pode ser usado na indexação')
print(a[multiplos_de_4])

print('\nOu diretamente:')
print(a[a > 4])

Valores
[[ 7 12  4  5]
 [ 1  6 11 10]
 [ 3  8  9  2]]

Resultado do teste item a item
[[False  True  True False]
 [False False False False]
 [False  True False False]]

... que pode ser usado na indexação
[12  4  8]

Ou diretamente:
[ 7 12  5  6 11 10  8  9]


## Pandas

[Pandas](https://pandas.pydata.org/) fornece implementações eficientes de estruturas de dados, especialmente [séries](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html#pandas.Series) e [dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame). A [documentação](https://pandas.pydata.org/pandas-docs/stable/) é bem completa, e pode-se ter boas noções em apenas [10 minutos](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

Algumas das principais vantagens dessa estrutura de dados são:
* indexação;
* ferramentas de:
  * manipulação de dados em memória e arquivo;
  * manipulação de valores ausentes;
  * manipulação de registros (colunas/linhas);
  * [agregação de dados](https://towardsdatascience.com/how-not-to-write-pandas-code-ef88599c6e8f).


# Pré-processamento

O pré-processamento dos dados geralmente é a parte mais trabalhosa do processo de mineração. Não só pelo esforço de formatar os dados para que possam ser aplicados em alguma ferramenta (como o tratamento de valores ausentes, normalização de valores, etc.), mas também pela busca pela [engenharia de características](https://heartbeat.fritz.ai/a-practical-guide-to-feature-engineering-in-python-8326e40747c8) mais adequada ao seu caso.

## Extração de dados

A primeira etapa do processo é a extração de dados dos sistemas de origem (bancos de dados, arquivos, sensores, etc.). No Kaggle, os arquivos são dados como [CSVs](https://pt.wikipedia.org/wiki/Comma-separated_values), portanto basta utilizar o comando [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

In [7]:
import pandas as pd

anoes = pd.read_csv('anoes.csv', index_col=0)
outros = pd.read_csv('outros.csv', index_col=0)

## Transformação de dados

A segunda etapa consiste em transformar os dados extraídos em um formato adequado para o processamento. Isso pode envolver uma série de passos, como:
* junção de dados de fontes distintas;
* seleção das característcas de interesse (ex: retirando dados pouco úteis);
* tradução de valores (ex: transformando valores categóricos em numéricos);
* derivação de características (ex: criar um novo valor área = base * altura);

In [8]:
# Junção de dados de fontes distintas;
data = pd.concat((anoes, outros), sort=False)

# Seleção das características de interesse
data.dropna(inplace=True)

# Derivação de características
data['Área'] = data.Largura * data.Comprimento

## Carregamento dos dados

A fase de carregamento consiste em armazenar os dados transformados para facilitar o acesso em proecssamentos futuros.

In [9]:
data.to_csv('data.csv')

Uma alternativa é usar o módulo [pickle](https://docs.python.org/3/library/pickle.html).

In [10]:
import pickle

with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

## Processamento

Com os dados devidamente organizados, pode-se começar a buscar as respostas a perguntas relevantes.

In [11]:
with open('data.pickle', 'rb') as f:
    df = pickle.load(f)

print(f'\n{df["Área"].idxmax()} é o maior latifundiário.')

futebol = df[((100 <= df.Largura) & (64 <= df.Comprimento)) | 
             ((100 <= df.Comprimento) & (64 <= df.Largura))]

#print(futebol)
nomes = futebol.index.tolist()
nomes = '{} e {}'.format(', '.join(nomes[:-1]), nomes[-1])
print(f'\n{nomes} podem ter campos de futebol em sua(s) propriedade(s).')


Príncipe é o maior latifundiário.

Atchim, Zangado e Príncipe podem ter campos de futebol em sua(s) propriedade(s).
